In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import hdf5getters as GETTERS
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.35:7077") \
        .appName("weak_scaling")\
        .config("spark.executor.memory", "2g")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", False) \
        .config("spark.shuffle.service.enabled", False) \
        .getOrCreate()
        
sc = spark_session.sparkContext

In [2]:
import io
import h5py
import math
import attr

def get_year(h5):
    return int(h5["musicbrainz"]["songs"][0][1])

def get_artist_hotness(h5):
    x = float(h5["metadata"]["songs"][0][3])
    if math.isnan(x):
        return 0.0
    else:
        return x
def get_song_hotness(h5):
    x =  float(h5["metadata"]["songs"][0][16])
    if math.isnan(x):
        return 0.0
    else:
        return x
def get_loudness(h5):
    x = float(h5["analysis"]["songs"][0][23])
    if math.isnan(x):
        return 0.0
    else:
        return x
def get_tempo(h5):
    x = float(h5["analysis"]["songs"][0][27])
    if math.isnan(x):
        return 0.0
    else:
        return x

def f(tup):
    #Function to extract necessary information for the analsis from raw data. 
    _path, data = tup
    with io.BytesIO(data) as b, h5py.File(b, "r") as f:
        return get_song_hotness(f), get_artist_hotness(f), get_loudness(f), get_tempo(f)

In [3]:
#Change the amount of data to be used by adding/removing letters (i.e. subdirectories) to/from these lists. 
#A_list as the lowest subdirectory in the tree, C_list the highest
import time

start_time = time.time() 

A_list = ["A"] #, "B", "C"]
B_list = ["A", "B","C","D","E","F","G","H","I","J","K","L","M"] #,"N","O", "P","Q", "R","S", "T","U","V","W","X","Y","Z"]
C_list = ["A", "B","C","D","E","F","G","H","I","J","K","L","M","N","O", "P","Q", "R","S", "T","U","V","W","X","Y","Z"]

#Generate paths in order to retrieve the data from hdfs

all_paths = ""
i = 0
for letter in A_list:
    path1 = "hdfs://192.168.2.35:9000/A/" + letter + "/"
    for letter in B_list:
        path2 = path1 + letter + "/"
        for letter in C_list:
            path3 = path2 + letter + "/*"
            if i == 0:
                all_paths = path3
                i += 1
                path3 = ""
            else:
                all_paths = all_paths + "," + path3
                path3 = ""
                
#Store the complete data in an RDD and extract the information needed for the analysis. 
#This includes song hotness, artist hotness, loudness and tempo (see functions defined above).

files = sc.binaryFiles(all_paths)
#counts = files.count()
rdd1 = files.map(lambda x: f(x))

In [6]:
from pyspark.mllib.stat import Statistics

#def correlation_analysis(perc_data):
#Function to do correlation analysis on a part of the dataset whose size is defined by perc_data

#rdd2 = rdd1.sample(None, perc_data)
rdd2 = rdd1.filter(lambda x: len(x) == 4)\
    .filter(lambda x: type(x[0])== float and type(x[1])== float and\
            type(x[2])== float and type(x[3])== float)

rdd_song_hotness = rdd2.map(lambda x: x[0])
rdd_artist_hotness = rdd2.map(lambda x: x[1])
rdd_loudness = rdd2.map(lambda x: x[2])
rdd_tempo = rdd2.map(lambda x: x[3])

corr_artist = Statistics.corr(rdd_song_hotness, rdd_artist_hotness, method="pearson")
corr_loudness = Statistics.corr(rdd_song_hotness, rdd_loudness, method="pearson")
corr_tempo = Statistics.corr(rdd_song_hotness, rdd_tempo, method="pearson")

print("Correlation song hotness and artist hotness: ", corr_artist,
"\nCorrelation song hotness and loudness: ", corr_loudness,
"\nCorrelation song hotness and tempo: ", corr_tempo)

end_time = time.time()

Py4JJavaError: An error occurred while calling o428.corr.
: java.net.ConnectException: Call From host-192-168-2-35-ldsa/192.168.2.35 to master-internal:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:792)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:732)
	at org.apache.hadoop.ipc.Client.call(Client.java:1480)
	at org.apache.hadoop.ipc.Client.call(Client.java:1413)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:229)
	at com.sun.

In [ ]:
execution_time1 #= end_time - start_time
execution_time2 #= end_time - start_time
execution_time3 = end_time - start_time
execution_time4 #= end_time - start_time

print(execution_time1, execution_time2, execution_time3, execution_time4)
sc.stop()

In [19]:
#Plotting the results of weak scalability test

import matplotlib
import matplotlib.pyplot as plt

plt.plot([round(counts/4), round(counts/2), round(3*counts/4), counts], [execution_time1, execution_time2, execution_time3, execution_time4])
plt.ylabel('Execution Time [s]')
plt.xlabel('Number of Files')
plt.show()
plt.savefig("weak_scalability")

<Figure size 640x480 with 1 Axes>

In [13]:
sc.stop()